# TABULAR PLAYGROUND SERIES OCT 2021

## ONLINE NOTEBOOK 
Stacking Ensemble Tutorial  
https://www.kaggle.com/code/junhyeok99/stacking-ensemble-tutorial

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
FOLDER_PATH = 'tps-oct-2021'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

TRAIN_PATH = os.path.join(FOLDER_PATH, TRAIN_FILE)
TEST_PATH = os.path.join(FOLDER_PATH, TEST_FILE)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [3]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

In [4]:
train_df.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,0.026177,0.106613,0.200924,0.713191,0.155750,0.557335,0.341702,0.285720,0.230396,0.203957,0.509588,0.706972,1,0.007793,0.247765,0.263750,0.259555,0.231730,0.138379,0.197824,0.054392,0.194153,0.281500,0.034818,0.025334,0.114432,0.139203,0.246157,0.251371,0.701423,0.301182,0.193924,0.267497,0,0.193430,0.238630,0.154770,0.249857,0.210685,0.406662,0.214810,0.258668,0.377518,0.192042,0.340855,0.199660,0.264074,0.205550,0.075109,0.205688,0.178962,0.245008,0.519336,0.306419,0.127139,0.367479,0.236380,0.195694,0.013195,0.199588,0.283367,0.168824,0.004855,0.117723,0.257688,0.197262,0.211452,0.372637,0.198157,0.689970,0.449955,0.713110,0.212041,0.183619,0.288667,0.648678,0.600398,0.223267,0.590163,0.248847,0.795641,0.139932,0.618696,0.639142,0.008549,0.559151,0.573640,0.138808,0.499156,0.112203,0.181498,0.165887,0.093171,0.106952,0.127861,0.250924,0.501673,0.036740,0.111361,0.075918,0.019444,0.250760,0.465093,0.087502,0.004185,0.195936,0.166389,0.171328,0.146014,0.199232,0.133999,0.168191,0.010242,0.294490,0.012977,0.003969,0.013739,0.040076,0.170711,0.250246,0.195538,0.708556,0.448925,0.550352,0.217984,0.751629,0.822459,0.186298,0.024197,0.044097,0.078943,0.181147,0.022591,0.576712,0.406843,0.510578,0.799434,0.651125,0.460708,0.636714,0.350704,0.872989,0.007751,0.017103,0.019875,0.203042,0.864594,0.595877,0.542969,0.990250,0.020373,0.006238,0.011040,0.006019,0.407014,0.080140,0.013502,0.144265,0.007229,0.003256,0.014556,0.123806,0.133871,0.011531,0.010250,0.253750,0.090162,0.147857,0.303087,0.112764,0.104344,0.168583,0.011342,0.239028,0.008018,0.167653,0.217342,0.184178,0.179060,0.078009,0.135768,0.006534,0.009832,0.013317,0.390079,0.004801,0.056600,0.114139,0.012599,0.014818,0.446073,0.216079,0.152113,0.111237,0.170896,0.190477,0.011936,0.005227,0.429740,0.013060,0.199369,0.258410,0.208863,0.129545,0.006978,0.012948,0.049466,0.008804,0.114205,0.119683,0.191210,0.169976,0.188199,0.355674,0.013164,0.304878,0.007213,0.011277,0.083186,0.010624,0.031199,0.200306,0.195791,0.203470,0.036314,0.157711,0.199117,0.007444,0.189048,0.202540,0.273267,0.167211,1,0,1,1,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1
1,1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,0.493411,0.107277,0.231828,0.457150,0.395200,0.617088,0.459358,0.209225,0.201098,0.199383,0.366578,0.585788,1,0.285311,0.400367,0.162493,0.249365,0.141160,0.133688,0.247906,0.139251,0.216444,0.109674,0.033018,0.017458,0.189336,0.168785,0.184251,0.202753,0.218451,0.324364,0.255499,0.287433,0,0.821982,0.284351,0.177537,0.207

In [5]:
test_df.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
0,1000000,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,0.343770,0.028081,0.102890,0.263804,0.607342,0.172180,0.491036,0.326189,0.252032,0.222362,0.242405,0.471047,0.723153,0,0.151190,0.226410,0.305478,0.269863,0.139583,0.141646,0.200469,0.059474,0.283620,0.244343,0.033877,0.010561,0.112109,0.193434,0.182685,0.188303,0.419009,0.285411,0.180912,0.209717,1,0.140470,0.241561,0.316918,0.215179,0.213195,0.246878,0.192177,0.178394,0.214424,0.182717,0.194714,0.174153,0.365876,0.169485,0.128054,0.250435,0.139776,0.268529,0.527465,0.338823,0.170422,0.163623,0.359837,0.149427,0.015599,0.162641,0.475365,0.218262,0.007640,0.113426,0.232048,0.196523,0.186975,0.323980,0.192931,0.634668,0.443279,0.687564,0.303014,0.209283,0.018238,0.581245,0.635117,0.164413,0.228907,0.489238,0.510888,0.651099,0.526209,0.542099,0.012908,0.150668,0.161797,0.271195,0.004828,0.113824,0.258710,0.283913,0.303373,0.105966,0.016048,0.030098,0.128585,0.038285,0.111251,0.154306,0.018076,0.209476,0.254074,0.084567,0.014150,0.350489,0.165743,0.169610,0.084159,0.187838,0.219015,0.168399,0.004914,0.019981,0.010063,0.004237,0.010785,0.043399,0.171178,0.291667,0.214174,0.709415,0.426936,0.576867,0.223034,0.719360,0.820207,0.185445,0.855089,0.533881,0.079138,0.625610,0.018667,0.623520,0.022663,0.365148,0.747389,0.558199,0.490998,0.729365,0.353262,0.875225,0.760399,0.414503,0.012137,0.203014,0.328036,0.548862,0.009272,0.997720,0.022934,0.004282,0.015800,0.896812,0.091122,0.079299,0.006342,0.145140,0.001878,0.002938,0.017270,0.235999,0.135111,0.635472,0.012371,0.016624,0.089912,0.105659,0.148205,0.002893,0.149323,0.169143,0.014200,0.195182,0.006760,0.250901,0.233035,0.095484,0.010419,0.078858,0.137215,0.009723,0.010456,0.008114,0.011521,0.012774,0.057635,0.401204,0.009198,0.008360,0.009917,0.162079,0.155008,0.059201,0.316937,0.095652,0.012927,0.117827,0.019541,0.015887,0.199542,0.317563,0.115914,0.162535,0.374658,0.013940,0.053678,0.004083,0.112384,0.011045,0.156239,0.154513,0.375630,0.006171,0.378678,0.006598,0.012125,0.004668,0.081293,0.010070,0.026948,0.433257,0.195952,0.227176,0.092359,0.152883,0.266000,0.006716,0.196959,0.022934,0.199017,0.150822,0,0,1,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,1,1,1,0
1,1000001,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,0.208373,0.278440,0.107586,0.402494,0.594466,0.193365,0.592103,0.345882,0.257781,0.201138,0.215895,0.449694,0.627330,1,0.133139,0.438526,0.215876,0.242926,0.156744,0.143899,0.697532,0.210877,0.268969,0.209255,0.040739,0.014843,0.113464,0.187680,0.206614,0.389319,0.535426,0.469284,0.525111,0.277706,0,0.816747,0.178317,0.255873,0.

In [6]:
for col in train_df.columns:
    if train_df[col].isna().sum():
        print(train_df[col].isna().sum())

if train_df.isna().sum().sum() == 0:
    print('There is no missing values in the training dataset.')

for col in test_df.columns:
    if test_df[col].isna().sum():
        print(test_df[col].isna().sum())
        
if test_df.isna().sum().sum() == 0:
    print('There is no missing valuse in the testing dataset.')

There is no missing values in the training dataset.
There is no missing valuse in the testing dataset.


In [7]:
y = train_df['target']

train_df = train_df.drop(columns=['id', 'target'])
test_df = test_df.drop(columns=['id'])

train_df['std'] = train_df.std(axis=1)
train_df['min'] = train_df.min(axis=1)
train_df['max'] = train_df.max(axis=1)

X = train_df

test_df['std'] = test_df.std(axis=1)
test_df['min'] = test_df.min(axis=1)
test_df['max'] = test_df.max(axis=1)

print(X.shape)
print(y.shape)
print(test_df.shape)

(1000000, 288)
(1000000,)
(500000, 288)


### MODELING
#### STACKING DATA LOADER

In [8]:
def Stacking_Data_Loader(model, model_name, X, y, test, fold):
    '''
    Function Parameters:
        model: model for training
        model_name: model name for training
        X: features of training dataset
        y: targets of training dataset
        test: features of testing dataset
        fold: number of folds
    
    Function return:
        train_fold_pred: predicted training targets
        test_pred_mean: mean of predicted testing targets over {fold} folds        
    '''
    
    skf = StratifiedKFold(n_splits=fold, random_state=42, shuffle=True)
    
    # Decalaration Prediction Datasets
    train_fold_pred = np.zeros((X.shape[0], 1))
    test_pred = np.zeros((test.shape[0], fold))
    
    for i, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_valid, y_valid = X.iloc[valid_idx, :], y[valid_idx]
        
        print(f'------------------- FOLD {i+1} Start -------------------')
        
        if model_name == 'cat':
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
        elif model_name == 'xgb':
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
                      verbose=500)
        else: #lightGBM
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
                      eval_metric='auc', verbose=500, early_stopping_rounds=200)
        
        print(f'------------------- FOLD {i+1} Done -------------------')
        
        train_fold_pred[valid_idx, :] = model.predict_proba(X_valid)[:, 1].reshape(-1, 1)
        test_pred[:, i] = model.predict_proba(test)[:, 1]
        
        del X_train, y_train, X_valid, y_valid
        gc.collect()
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    del test_pred
    gc.collect()
    
    print('Done!')
    
    return train_fold_pred, test_pred_mean

#### HYPER PARAMETERS

In [9]:
lgb_params = {
    'objective': 'binary', 
    'n_estimators': 20_000, 
    'random_state': 42, 
    'learning_rate': 8e-3, 
    'subsample': 0.6, 
    'subsample_freq': 1, 
    'colsample_bytree': 0.4, 
    'reg_alpha': 10.0, 
    'reg_lambda': 1e-1, 
    'min_child_weight': 256, 
    'min_child_samples': 20, 
    #'device': 'gpu', 
}

xgb_params = {
    'n_estimators': 10_000, 
    'learning_rate': 0.03689407512484644, 
    'max_depth': 8, 
    'colsample_bytree': 0.3723914688159835, 
    'subsample': 0.780714581166012, 
    'eval_metric': 'auc', 
    'use_label_encoder': False, 
    'gamma': 0, 
    'reg_lambda': 50.0, 
    'tree_method': 'gpu_hist', 
    'gpu_id': 0, 
    'predictor': 'gpu_predictor', 
    'random_state': 42, 
    'early_stopping_rounds': 200, 
}

cat_params = {
    'iterations': 17298, 
    'learning_rate': 0.03429054860458741, 
    'reg_lambda': 0.3242286463210283, 
    'subsample': 0.9433911589913944, 
    'random_strength': 22.4849972385133, 
    'depth': 8, 
    'min_data_in_leaf': 4, 
    'leaf_estimation_iterations': 8, 
    'task_type': 'GPU', 
    'bootstrap_type': 'Poisson', 
    'verbose': 500, 
    'early_stopping_rounds': 200, 
    'eval_metric': 'AUC', 
}

#### STACKING

In [10]:
lgbm = LGBMClassifier(**lgb_params)
xgb = XGBClassifier(**xgb_params)
cat = CatBoostClassifier(**cat_params)

In [11]:
%%time 

cat_train, cat_test = Stacking_Data_Loader(cat, 'cat', X, y, test_df, 5)
del cat
gc.collect()

------------------- FOLD 1 Start -------------------


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8035762	best: 0.8035762 (0)	total: 78.8ms	remaining: 22m 43s
500:	test: 0.8552158	best: 0.8552158 (500)	total: 37.3s	remaining: 20m 52s
1000:	test: 0.8574535	best: 0.8574535 (1000)	total: 1m 17s	remaining: 21m 2s
1500:	test: 0.8579972	best: 0.8580008 (1495)	total: 1m 56s	remaining: 20m 25s
2000:	test: 0.8581418	best: 0.8581527 (1968)	total: 2m 31s	remaining: 19m 15s
2500:	test: 0.8581675	best: 0.8581848 (2428)	total: 3m 5s	remaining: 18m 15s
bestTest = 0.8581847548
bestIteration = 2428
Shrink model to first 2429 iterations.
------------------- FOLD 1 Done -------------------
------------------- FOLD 2 Start -------------------


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7991146	best: 0.7991146 (0)	total: 104ms	remaining: 29m 59s
500:	test: 0.8514802	best: 0.8514802 (500)	total: 36.4s	remaining: 20m 21s
1000:	test: 0.8542027	best: 0.8542027 (1000)	total: 1m 11s	remaining: 19m 31s
1500:	test: 0.8548059	best: 0.8548066 (1499)	total: 1m 46s	remaining: 18m 39s
2000:	test: 0.8550975	best: 0.8551005 (1996)	total: 2m 20s	remaining: 17m 55s
2500:	test: 0.8551775	best: 0.8551788 (2421)	total: 2m 55s	remaining: 17m 16s
3000:	test: 0.8552096	best: 0.8552203 (2965)	total: 3m 29s	remaining: 16m 37s
bestTest = 0.8552202582
bestIteration = 2965
Shrink model to first 2966 iterations.
------------------- FOLD 2 Done -------------------
------------------- FOLD 3 Start -------------------


...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7988047	best: 0.7988047 (0)	total: 103ms	remaining: 29m 43s
500:	test: 0.8524361	best: 0.8524393 (498)	total: 35.6s	remaining: 19m 53s
1000:	test: 0.8548340	best: 0.8548340 (1000)	total: 1m 10s	remaining: 19m 3s
1500:	test: 0.8553504	best: 0.8553504 (1500)	total: 1m 43s	remaining: 18m 13s
2000:	test: 0.8554510	best: 0.8554575 (1955)	total: 2m 16s	remaining: 17m 27s
bestTest = 0.8554711938
bestIteration = 2286
Shrink model to first 2287 iterations.
------------------- FOLD 4 Done -------------------
------------------- FOLD 5 Start -------------------


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8002287	best: 0.8002287 (0)	total: 96.7ms	remaining: 27m 52s
500:	test: 0.8531870	best: 0.8531870 (500)	total: 35.8s	remaining: 19m 59s
1000:	test: 0.8554730	best: 0.8554730 (1000)	total: 1m 10s	remaining: 19m 7s
1500:	test: 0.8559324	best: 0.8559330 (1497)	total: 1m 43s	remaining: 18m 14s
2000:	test: 0.8560385	best: 0.8560467 (1951)	total: 2m 17s	remaining: 17m 29s
2500:	test: 0.8560781	best: 0.8561049 (2395)	total: 2m 50s	remaining: 16m 46s
bestTest = 0.8561048508
bestIteration = 2395
Shrink model to first 2396 iterations.
------------------- FOLD 5 Done -------------------
Done!
CPU times: total: 34min 56s
Wall time: 16min 23s


0

In [12]:
%%time

lgbm_train, lgbm_test = Stacking_Data_Loader(lgbm, 'lgbm', X, y, test_df, 5)
del lgbm
gc.collect()

------------------- FOLD 1 Start -------------------
Training until validation scores don't improve for 200 rounds
[500]	valid_0's auc: 0.84583	valid_0's binary_logloss: 0.502928
[1000]	valid_0's auc: 0.850332	valid_0's binary_logloss: 0.482302
[1500]	valid_0's auc: 0.853622	valid_0's binary_logloss: 0.47536
[2000]	valid_0's auc: 0.855558	valid_0's binary_logloss: 0.471392
[2500]	valid_0's auc: 0.856739	valid_0's binary_logloss: 0.468828
[3000]	valid_0's auc: 0.857486	valid_0's binary_logloss: 0.467113
[3500]	valid_0's auc: 0.857984	valid_0's binary_logloss: 0.465934
[4000]	valid_0's auc: 0.858312	valid_0's binary_logloss: 0.465104
[4500]	valid_0's auc: 0.858518	valid_0's binary_logloss: 0.464522
[5000]	valid_0's auc: 0.85866	valid_0's binary_logloss: 0.464105
[5500]	valid_0's auc: 0.858748	valid_0's binary_logloss: 0.463823
[6000]	valid_0's auc: 0.858791	valid_0's binary_logloss: 0.463631
[6500]	valid_0's auc: 0.858835	valid_0's binary_logloss: 0.463482
[7000]	valid_0's auc: 0.858859	

0

In [13]:
%%time

xgb_train, xgb_test = Stacking_Data_Loader(xgb, 'xgb', X, y, test_df, 5)
del xgb
gc.collect()

------------------- FOLD 1 Start -------------------
[0]	validation_0-auc:0.81212
[500]	validation_0-auc:0.85695
[1000]	validation_0-auc:0.85787
[1304]	validation_0-auc:0.85786
------------------- FOLD 1 Done -------------------
------------------- FOLD 2 Start -------------------
[0]	validation_0-auc:0.80775
[500]	validation_0-auc:0.85374
[1000]	validation_0-auc:0.85501
[1314]	validation_0-auc:0.85504
------------------- FOLD 2 Done -------------------
------------------- FOLD 3 Start -------------------
[0]	validation_0-auc:0.80937
[500]	validation_0-auc:0.85533
[1000]	validation_0-auc:0.85665
[1229]	validation_0-auc:0.85666
------------------- FOLD 3 Done -------------------
------------------- FOLD 4 Start -------------------
[0]	validation_0-auc:0.80789
[500]	validation_0-auc:0.85430
[1000]	validation_0-auc:0.85528
[1314]	validation_0-auc:0.85522
------------------- FOLD 4 Done -------------------
------------------- FOLD 5 Start -------------------
[0]	validation_0-auc:0.80882
[5

0

#### FINAL STACKING DATASET

In [14]:
stack_X_train = np.concatenate((cat_train, lgbm_train, xgb_train), axis=1)
stack_X_test = np.concatenate((cat_test, lgbm_test, xgb_test), axis=1)

del cat_train, lgbm_train, xgb_train, cat_test, lgbm_test, xgb_test
gc.collect()

stack_X_train

array([[0.65206619, 0.64492836, 0.65455836],
       [0.19428487, 0.18877744, 0.20673431],
       [0.8451595 , 0.86497772, 0.86749905],
       ...,
       [0.16366753, 0.20006934, 0.20250763],
       [0.90414119, 0.92206844, 0.90553403],
       [0.58451988, 0.54142374, 0.50090343]])

### LEVEL 2 TRAINING

In [19]:
skf = StratifiedKFold(n_splits=5)

test_pred_lo = 0
fold = 1
total_auc = 0

for train_idx, valid_idx in skf.split(stack_X_train, y):
    X_train, y_train = stack_X_train[train_idx], y[train_idx]
    X_valid, y_valid = stack_X_train[valid_idx], y[valid_idx]
    
    lr = LogisticRegression(n_jobs=-1, random_state=42, C=5, max_iter=2_000)
    lr.fit(X_train, y_train)
    
    valid_pred_lo = lr.predict_proba(X_valid)[:, 1]
    test_pred_lo += lr.predict_proba(stack_X_test)[:, 1] / 5
    auc = roc_auc_score(y_valid, valid_pred_lo)
    total_auc += auc / 5
    print(f'Fold: {fold}, AUC: {auc}')
    fold += 1

print(f'Total AUC Score: {total_auc}.')
print('----------------TEST DATASET----------------')
print(f'The max probability is {max(test_pred_lo)}.')
print(f'The min probability is {min(test_pred_lo)}.')

Fold: 1, AUC: 0.857009277260029
Fold: 2, AUC: 0.8595812012799522
Fold: 3, AUC: 0.8597383090277749
Fold: 4, AUC: 0.8552100213671092
Fold: 5, AUC: 0.8553381965877092
Total AUC Score: 0.8573754011045149.
----------------TEST DATASET----------------
The max probability is 0.9345118861151729.
The min probability is 0.0709143943367378.


## ONLINE NOTEBOOK 2 
TPS Oct 2021 - PCA and KMeans (Feature Eng.)  
https://www.kaggle.com/code/kavehshahhosseini/tps-oct-2021-pca-and-kmeans-feature-eng

In [23]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_regression

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import shap

In [25]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)